# Quadrant Classification (Random Search Training)

This notebook:
- Creates a 2D dataset labeled by quadrant (4 classes)
- Visualizes the dataset
- Defines a tiny neural net (2 → 8 → 4) with ReLU + Softmax
- Trains using a simple random-search / hill-climb on weights and biases


In [34]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)


In [35]:
def createDataset(points, classes=4):
    X = np.random.uniform(-1, 1, size=(points * classes, 2))
    y = np.zeros(points * classes, dtype='uint8')

    for i, (x, y_val) in enumerate(X):
        if x >= 0 and y_val >= 0:
            y[i] = 0  # Quad 1
        elif x < 0 and y_val >= 0:
            y[i] = 1  # Quad 2
        elif x < 0 and y_val < 0:
            y[i] = 2  # Quad 3
        else:
            y[i] = 3  # Quad 4

    return X, y
X,y = createDataset(1000,4)


## Model components

A minimal dense layer, ReLU, Softmax, and Categorical Cross-Entropy loss.


In [36]:
class Neuron_Layer():
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases


class Activation_ReLU():
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)


class Activation_Softmax():
    def forward(self, inputs):
        exps = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        self.output = exps / np.sum(exps, axis=1, keepdims=True)


class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        return np.mean(sample_losses)


class Loss_CategoricalCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        return -np.log(correct_confidences)


def accuracy(y_pred, y_true):
    predictions = np.argmax(y_pred, axis=1)
    return np.mean(predictions == y_true)


## Initialize network

Network: 2 → 8 → 4 (ReLU then Softmax)


In [37]:
# Initial random weights
w1 = 0.1 * np.random.randn(2, 8)
w2 = 0.1 * np.random.randn(8, 4)

# Best weights so far (start with initial)
bw1 = w1.copy()
bw2 = w2.copy()

bloss = 9_999_999

# Build network
layer1 = Neuron_Layer(2, 8)
layer1.weights = w1.copy()
activation1 = Activation_ReLU()

layer2 = Neuron_Layer(8, 4)
layer2.weights = w2.copy()
activation2 = Activation_Softmax()

# Best biases so far
bb1 = layer1.biases.copy()
bb2 = layer2.biases.copy()


## Training loop (random search / hill climbing)

At each iteration:
- Propose slightly perturbed weights/biases
- Keep the change if loss improves, otherwise revert


In [38]:
loss_fn = Loss_CategoricalCrossentropy()

for i in range(50001):
    # Propose new parameters around best known
    nw1 = bw1 + 0.0005 * np.random.randn(2, 8)
    nw2 = bw2 + 0.0005 * np.random.randn(8, 4)

    nb1 = bb1 + 0.0005 * np.random.randn(1, 8)
    nb2 = bb2 + 0.0005 * np.random.randn(1, 4)

    # Assign proposal
    layer1.weights = nw1
    layer2.weights = nw2
    layer1.biases = nb1
    layer2.biases = nb2

    # Forward pass
    layer1.forward(X)
    activation1.forward(layer1.output)
    layer2.forward(activation1.output)
    activation2.forward(layer2.output)

    # Evaluate
    los = loss_fn.calculate(activation2.output, y)
    acc = accuracy(activation2.output, y)

    if i % 5000 == 0:
        print(f"Iter {i:6d} | loss: {los:.4f} | acc: {acc:.4f}")

    # Accept/reject
    if los < bloss:
        bw1, bw2 = nw1, nw2
        bb1, bb2 = nb1, nb2
        bloss = los
    else:
        # revert
        layer1.weights = bw1
        layer2.weights = bw2
        layer1.biases = bb1
        layer2.biases = bb2


Iter      0 | loss: 1.3805 | acc: 0.4915
Iter   5000 | loss: 1.2215 | acc: 0.7348
Iter  10000 | loss: 0.9698 | acc: 0.8073
Iter  15000 | loss: 0.6862 | acc: 0.9030
Iter  20000 | loss: 0.4624 | acc: 0.9563
Iter  25000 | loss: 0.3079 | acc: 0.9690
Iter  30000 | loss: 0.2120 | acc: 0.9822
Iter  35000 | loss: 0.1544 | acc: 0.9885
Iter  40000 | loss: 0.1161 | acc: 0.9922
Iter  45000 | loss: 0.0916 | acc: 0.9958
Iter  50000 | loss: 0.0741 | acc: 0.9958


## Final evaluation


In [39]:
layer1.forward(X)
activation1.forward(layer1.output)
layer2.forward(activation1.output)
activation2.forward(layer2.output)

final_loss = loss_fn.calculate(activation2.output, y)
final_acc = accuracy(activation2.output, y)

print("Final loss:", final_loss)
print("Final acc:", final_acc)


Final loss: 0.07409305430574245
Final acc: 0.99575
